# TiledCopy内存拷贝 - 实践篇

本notebook帮助你理解cute的TiledCopy抽象及其在FlashAttention中的应用。

**学习目标：**
- 理解GPU内存层次和数据传输
- 理解TiledCopy的组成和使用
- 分析异步拷贝流水线的原理


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 1. GPU内存层次可视化


In [ ]:
def visualize_memory_hierarchy():
    """可视化GPU内存层次"""
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    # 寄存器
    rect1 = plt.Rectangle((2, 7), 6, 1.5, fill=True, 
                           facecolor='lightcoral', edgecolor='darkred', linewidth=2)
    ax.add_patch(rect1)
    ax.text(5, 7.75, '寄存器 (Registers)\n最快 | 每线程私有 | MMA输入输出', 
            ha='center', va='center', fontsize=10)
    
    # 共享内存
    rect2 = plt.Rectangle((2, 4.5), 6, 1.5, fill=True, 
                           facecolor='lightyellow', edgecolor='orange', linewidth=2)
    ax.add_patch(rect2)
    ax.text(5, 5.25, '共享内存 (SMEM)\n~20周期延迟 | 线程块共享 | 数据暂存', 
            ha='center', va='center', fontsize=10)
    
    # 全局内存
    rect3 = plt.Rectangle((2, 2), 6, 1.5, fill=True, 
                           facecolor='lightblue', edgecolor='darkblue', linewidth=2)
    ax.add_patch(rect3)
    ax.text(5, 2.75, '全局内存 (HBM)\n~400周期延迟 | 所有线程 | 输入输出数据', 
            ha='center', va='center', fontsize=10)
    
    # 箭头和标签
    ax.annotate('', xy=(5, 7), xytext=(5, 6.2),
                arrowprops=dict(arrowstyle='<->', color='green', lw=2))
    ax.text(7, 6.6, 'S2R / R2S\n(ldmatrix)', ha='left', fontsize=9, color='green')
    
    ax.annotate('', xy=(5, 4.5), xytext=(5, 3.7),
                arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    ax.text(7, 4.1, 'G2S / S2G\n(cp.async)', ha='left', fontsize=9, color='blue')
    
    # 性能数据
    ax.text(0.5, 0.8, 'A100 性能: HBM带宽 2TB/s, SMEM带宽 ~19TB/s', fontsize=10)
    ax.text(0.5, 0.4, 'TiledCopy的作用: 封装这些传输操作，优化带宽利用', fontsize=10, style='italic')
    
    ax.set_title('GPU内存层次与TiledCopy', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_memory_hierarchy()


## 2. TiledCopy的线程分配


In [ ]:
def visualize_thread_layout():
    """可视化TiledCopy的线程分配"""
    # 假设 ThreadLayout = (16, 8), ValueLayout = (1, 8)
    # 每个线程处理 1×8 = 8 个元素
    # 总tile: 16×64
    
    thread_layout = (16, 8)  # 16行8列的线程
    value_layout = (1, 8)    # 每线程1行8列
    
    tile_m = thread_layout[0] * value_layout[0]  # 16
    tile_n = thread_layout[1] * value_layout[1]  # 64
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 左图: 线程布局
    ax = axes[0]
    thread_grid = np.arange(thread_layout[0] * thread_layout[1]).reshape(thread_layout)
    im = ax.imshow(thread_grid, cmap='tab20', vmin=0, vmax=127)
    ax.set_title(f'ThreadLayout: {thread_layout[0]}×{thread_layout[1]}=128线程')
    ax.set_xlabel('线程列')
    ax.set_ylabel('线程行')
    
    for i in range(thread_layout[0]):
        for j in range(thread_layout[1]):
            ax.text(j, i, f'T{thread_grid[i,j]}', ha='center', va='center', 
                   fontsize=6, color='white')
    
    plt.colorbar(im, ax=ax, label='线程ID')
    
    # 右图: 数据分配
    ax = axes[1]
    data_assign = np.zeros((tile_m, tile_n), dtype=int)
    for ti in range(thread_layout[0]):
        for tj in range(thread_layout[1]):
            thread_id = ti * thread_layout[1] + tj
            for vi in range(value_layout[0]):
                for vj in range(value_layout[1]):
                    di = ti * value_layout[0] + vi
                    dj = tj * value_layout[1] + vj
                    data_assign[di, dj] = thread_id
    
    im = ax.imshow(data_assign, cmap='tab20', vmin=0, vmax=127)
    ax.set_title(f'数据Tile: {tile_m}×{tile_n}\\n每个颜色=一个线程负责的区域')
    ax.set_xlabel('列')
    ax.set_ylabel('行')
    plt.colorbar(im, ax=ax, label='负责的线程ID')
    
    plt.suptitle('TiledCopy线程到数据的映射', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"ThreadLayout: {thread_layout} = {thread_layout[0]*thread_layout[1]} 线程")
    print(f"ValueLayout: {value_layout} = 每线程 {value_layout[0]*value_layout[1]} 元素")
    print(f"总Tile大小: {tile_m}×{tile_n} = {tile_m*tile_n} 元素")
    print(f"每线程处理: {tile_m*tile_n//(thread_layout[0]*thread_layout[1])} 元素")

visualize_thread_layout()


## 3. 异步拷贝流水线


In [ ]:
def visualize_pipeline():
    """可视化异步拷贝流水线"""
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # 时间轴
    num_iters = 6
    stages = 2
    
    y_positions = {'Load Stage 0': 3, 'Load Stage 1': 2, 'Compute': 1}
    colors = {'Load Stage 0': 'lightblue', 'Load Stage 1': 'lightgreen', 'Compute': 'lightyellow'}
    
    for i in range(num_iters):
        stage = i % stages
        
        # 加载操作
        load_y = y_positions[f'Load Stage {stage}']
        load_rect = plt.Rectangle((i*1.5, load_y - 0.3), 1.2, 0.6, 
                                   facecolor=colors[f'Load Stage {stage}'], 
                                   edgecolor='black')
        ax.add_patch(load_rect)
        ax.text(i*1.5 + 0.6, load_y, f'Load K{i}', ha='center', va='center', fontsize=9)
        
        # 计算操作 (延迟一个迭代)
        if i > 0:
            compute_rect = plt.Rectangle((i*1.5, y_positions['Compute'] - 0.3), 1.2, 0.6, 
                                         facecolor=colors['Compute'], 
                                         edgecolor='black')
            ax.add_patch(compute_rect)
            ax.text(i*1.5 + 0.6, y_positions['Compute'], f'MMA K{i-1}', 
                   ha='center', va='center', fontsize=9)
    
    # 标签
    ax.text(-0.5, 3, 'Stage 0', ha='right', va='center', fontsize=10)
    ax.text(-0.5, 2, 'Stage 1', ha='right', va='center', fontsize=10)
    ax.text(-0.5, 1, '计算', ha='right', va='center', fontsize=10)
    
    ax.set_xlim(-1, num_iters * 1.5 + 0.5)
    ax.set_ylim(0, 4)
    ax.set_xlabel('时间 →', fontsize=12)
    ax.set_title('双缓冲异步拷贝流水线\\n加载和计算重叠，隐藏内存延迟', fontsize=14)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("流水线关键点:")
    print("1. cp.async 发起异步拷贝，不等待完成")
    print("2. cp_async_fence() 标记一组拷贝")
    print("3. cp_async_wait<N>() 等待直到只剩N组未完成")
    print("4. 双缓冲: 一边加载下一块，一边计算当前块")

visualize_pipeline()


## 4. FlashAttention中的TiledCopy代码分析


In [ ]:
tiledcopy_code = '''
// ============================================================
// FlashAttention中的TiledCopy使用模式
// 来自 hopper/mainloop_fwd_sm80.hpp
// ============================================================

// 1. 定义G2S TiledCopy (全局内存 → 共享内存)
using GmemTiledCopyQKV = TiledCopy<
    Copy_Atom<SM80_CP_ASYNC_CACHEGLOBAL<cute::uint128_t>, Element>,
    Layout<Shape<_16, _8>>,    // 16×8 线程布局
    Layout<Shape<_1, _8>>      // 每线程处理 1×8
>;

// 2. 创建TiledCopy并获取线程视图
GmemTiledCopyQKV gmem_tiled_copy_QKV;
auto gmem_thr_copy_QKV = gmem_tiled_copy_QKV.get_thread_slice(threadIdx.x);

// 3. 分区源和目标Tensor
Tensor tQgQ = gmem_thr_copy_QKV.partition_S(gQ);  // 全局内存Q
Tensor tQsQ = gmem_thr_copy_QKV.partition_D(sQ);  // 共享内存Q

// 4. 执行异步拷贝
cute::copy(gmem_tiled_copy_QKV, tQgQ, tQsQ);
cute::cp_async_fence();

// 5. 等待拷贝完成
cute::cp_async_wait<0>();
__syncthreads();

// ============================================================
// S2R拷贝 (共享内存 → 寄存器，为MMA准备)
// ============================================================

// 创建与MMA配合的SMEM拷贝
auto smem_tiled_copy_A = make_tiled_copy_A(SmemCopyAtom{}, tiled_mma);
auto smem_thr_copy_A = smem_tiled_copy_A.get_thread_slice(threadIdx.x);

// 分区
Tensor tCsQ = smem_thr_copy_A.partition_S(sQ);
Tensor tCrQ = smem_thr_copy_A.retile_D(tCrQ_mma);  // retile匹配MMA布局

// 拷贝到寄存器
cute::copy(smem_tiled_copy_A, tCsQ(_, _, k), tCrQ(_, _, k));
'''

print(tiledcopy_code)


## 5. 数据流模拟


In [ ]:
def simulate_data_flow():
    """模拟FlashAttention的数据流"""
    print("FlashAttention 数据流")
    print("=" * 60)
    
    # 配置
    seqlen = 2048
    headdim = 64
    kBlockM = 128
    kBlockN = 64
    
    num_q_blocks = seqlen // kBlockM
    num_kv_blocks = seqlen // kBlockN
    
    print(f"配置: seqlen={seqlen}, headdim={headdim}")
    print(f"      kBlockM={kBlockM}, kBlockN={kBlockN}")
    print()
    
    # 模拟一个block的数据流
    print("单个Thread Block的数据流:")
    print("-" * 60)
    
    print("""
1. 加载Q块 (一次性)
   GMEM → SMEM: TiledCopy(gmem_tiled_copy_Q)
   ┌──────────────────────────────┐
   │  Q[m_block*kBlockM : (m_block+1)*kBlockM, :headdim]  │
   │  大小: 128×64 = 8192 元素 = 16KB (FP16)              │
   └──────────────────────────────┘

2. 循环处理K/V块
   for n_block in range(num_kv_blocks):
   
       a. 加载K块
          GMEM → SMEM: TiledCopy(gmem_tiled_copy_KV)
          ┌──────────────────────────────┐
          │  K[n_block*kBlockN : (n_block+1)*kBlockN, :]  │
          │  大小: 64×64 = 4096 元素 = 8KB (FP16)         │
          └──────────────────────────────┘
          
       b. 加载V块 (类似K)
       
       c. 同步等待拷贝完成
          cp_async_wait<0>()
          __syncthreads()
       
       d. S2R拷贝: SMEM → Register
          TiledCopy(smem_tiled_copy)
          
       e. MMA计算: S = Q @ K^T
          TiledMMA执行
          
       f. Softmax计算 (在寄存器中)
       
       g. MMA计算: O += P @ V

3. 写回输出
   Register → SMEM → GMEM
""")

simulate_data_flow()


## 总结

通过本notebook，你应该理解了：

1. **GPU内存层次**: GMEM → SMEM → Register 的数据路径
2. **TiledCopy组成**: Copy Atom + ThreadLayout + ValueLayout
3. **异步拷贝**: cp.async 指令，支持流水线执行
4. **partition操作**: 将Tensor分配给线程
5. **FlashAttention应用**: G2S加载Q/K/V，S2R准备MMA输入

## 关键代码模式

```cpp
// G2S: 全局内存 → 共享内存
auto thr_copy = gmem_tiled_copy.get_thread_slice(threadIdx.x);
Tensor tSgS = thr_copy.partition_S(gS);  // 源
Tensor tSsS = thr_copy.partition_D(sS);  // 目标
cute::copy(gmem_tiled_copy, tSgS, tSsS);
cute::cp_async_fence();
cute::cp_async_wait<0>();

// S2R: 共享内存 → 寄存器
auto smem_thr = smem_tiled_copy.get_thread_slice(threadIdx.x);
Tensor tCsS = smem_thr.partition_S(sS);
Tensor tCrS = smem_thr.retile_D(tCrS_mma);
cute::copy(smem_tiled_copy, tCsS, tCrS);
```

## 下一步

在下一节"local_tile局部分块"中，我们将学习如何从大Tensor中获取分块视图。
